## モジュールのインポート

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

## 変数定義

In [2]:
kaisai_date = "20260228"
year = kaisai_date[:4]

DATA_DIR = Path(".", "data", year, kaisai_date)
DATA_DIR.mkdir(exist_ok=True, parents=True)

KAKO_DATA_DIR = Path("C:/", "TFJV", "TXT", "data", "kako_data", year)
BASE_DATA_DIR = Path("C:/", "TFJV", "TXT", "data", "base_data", year)
PEDS_DATA_DIR = Path("C:/", "TFJV", "TXT", "data", "peds_data", year)

## ファイルの読み込み

In [3]:
zensou_df = pd.read_csv(KAKO_DATA_DIR / f"kako_data_{kaisai_date}.csv", encoding="shift-jis", header=None)
base_df = pd.read_csv(BASE_DATA_DIR / f"base_data_{kaisai_date}.csv", encoding="shift-jis", header=None)
peds_df = pd.read_csv(PEDS_DATA_DIR / f"peds_data_{kaisai_date}.csv", encoding="shift-jis", header=None)

In [4]:
zensou_df_pre = zensou_df.iloc[:, [8, 21, 25, 30, 43, 64]]
# base_df_pre = base_df.iloc[:, [3, 4, 5, 7, 8, 9, 14, 21, 24, 26, 33]]
# peds_df_pre = peds_df.iloc[:, [21, 32]]

# 前日などにCSVをダウンロードした時はこちらを使う
base_df_pre = base_df.iloc[:, [3, 4, 5, 7, 8, 9, 14, 21, 24, 26, 31]]
peds_df_pre = peds_df.iloc[:, [21, 30]]

## 前処理

### 前走データ

In [5]:
# カラム名を付与
zensou_df_pre.columns = ["距離", "馬名", "前走間隔", "前走距離", "前-2走前間隔", "2-3走前間隔"]

In [6]:
# 文字列型データを数値型データに変換
zensou_df_pre = zensou_df_pre.replace("連", 1)
zensou_df_pre = zensou_df_pre.replace("初", 0)

zensou_df_pre["前走間隔"] = pd.to_numeric(zensou_df_pre["前走間隔"], errors="coerce").astype(float)
zensou_df_pre["前-2走前間隔"] = pd.to_numeric(zensou_df_pre["前-2走前間隔"], errors="coerce").astype(float)
zensou_df_pre["2-3走前間隔"] = pd.to_numeric(zensou_df_pre["2-3走前間隔"], errors="coerce").astype(float)

In [7]:
int_cols = zensou_df_pre.select_dtypes(include=['int']).columns
zensou_df_pre[int_cols] = zensou_df_pre[int_cols].astype('float')

In [8]:
# 「中○週」の数値に変換
zensou_df_pre["前走間隔"] = zensou_df_pre["前走間隔"] - 1
zensou_df_pre["前-2走前間隔"] = zensou_df_pre["前-2走前間隔"] - 1
zensou_df_pre["2-3走前間隔"] = zensou_df_pre["2-3走前間隔"] - 1

In [9]:
# 「臨戦過程」の列を追加
import numpy as np

l = []
for i in range(len(zensou_df_pre)):
    if zensou_df_pre.loc[i, "前走間隔"] == 0:
        l.append("連闘")
    elif zensou_df_pre.loc[i, "前走間隔"] >= 12:
        l.append("休明初戦")
    elif zensou_df_pre.loc[i, "前-2走前間隔"] >= 12:
        l.append("休明2走")
    elif zensou_df_pre.loc[i, "2-3走前間隔"] >= 12:
        l.append("休明3走")
    else:
        l.append(np.nan)

zensou_df_pre["臨戦過程"] = l

In [10]:
# 「距離変遷」の列を追加
n = []
for i in range(len(zensou_df_pre)):
    if zensou_df_pre.loc[i, "前走距離"] - zensou_df_pre.loc[i, "距離"] > 0:
        n.append("距離短縮")
    elif zensou_df_pre.loc[i, "前走距離"] - zensou_df_pre.loc[i, "距離"] < 0:
        n.append("距離延長")
    else:
        n.append(np.nan)

zensou_df_pre["距離変遷"] = n

In [11]:
# 使用するカラムを絞る
zensou_df_pre = zensou_df_pre[[
    "馬名",
    "臨戦過程",
    "距離変遷"
]]

### 基本データ

In [12]:
# カラム名を付与
base_df_pre.columns = [
    "場所",
    "R",
    "クラス",
    "種別",
    "距離",
    "馬場状態",
    "馬番",
    "馬名",
    "年齢",
    "騎手名",
    "調教師名"
]

In [13]:
# 「クラス」列のデータを置換
base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*未勝利.*", "未勝利", regex=True)
base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*新馬.*", "新馬", regex=True)
base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*(１勝|２勝|３勝|1勝|2勝|3勝).*", "自己条件", regex=True)
base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*(G1|G2|G3).*", "重賞", regex=True)

condition = ~base_df_pre['クラス'].str.contains('新馬|未勝利|自己条件|重賞')
# base_df_pre.loc[condition, 'クラス'] = base_df_pre['クラス'].loc[condition].str.replace(r".*", "OP", regex=True)
base_df_pre.loc[condition, 'クラス'] = "OP"

C:\Users\yf_te\AppData\Local\Temp\ipykernel_1492\3848943296.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*未勝利.*", "未勝利", regex=True)
C:\Users\yf_te\AppData\Local\Temp\ipykernel_1492\3848943296.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*新馬.*", "新馬", regex=True)
C:\Users\yf_te\AppData\Local\Temp\ipykernel_1492\3848943296.py:4: SettingWithCopyWarning: 
A value is trying to be set on a 

In [14]:
# 距離区分の列を追加
l = []
for i in range(len(base_df_pre)):
    if base_df_pre.loc[i, "距離"] <= 1600:
        l.append("短距離")
    elif base_df_pre.loc[i, "距離"] <= 2200:
        l.append("中距離")
    else:
        l.append("長距離")

base_df_pre["距離区分"] = l

C:\Users\yf_te\AppData\Local\Temp\ipykernel_1492\19989305.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df_pre["距離区分"] = l


In [15]:
# 回りの区分の列を追加
l = []
left_tern = re.compile('東京|新潟|中京')

for i in range(len(base_df_pre)):
    if left_tern.search(base_df_pre.loc[i, "場所"]):
        l.append("左回り")
    else:
        l.append("右回り")

base_df_pre["回り"] = l

C:\Users\yf_te\AppData\Local\Temp\ipykernel_1492\789649226.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df_pre["回り"] = l


In [16]:
# 年齢データを変換
for i in range(2, 14):
    if i <= 4:
        base_df_pre['年齢'] = base_df_pre['年齢'].replace(i, f"{i}歳")
    else:
        base_df_pre['年齢'] = base_df_pre['年齢'].replace(i, "5歳以上")

C:\Users\yf_te\AppData\Local\Temp\ipykernel_1492\569448488.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df_pre['年齢'] = base_df_pre['年齢'].replace(i, f"{i}歳")
C:\Users\yf_te\AppData\Local\Temp\ipykernel_1492\569448488.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df_pre['年齢'] = base_df_pre['年齢'].replace(i, "5歳以上")


In [17]:
base_df_pre["年齢"].value_counts()

年齢
3歳      240
5歳以上    164
4歳      116
Name: count, dtype: int64

In [18]:
# 馬場状態（道悪かどうか）の判定列を作る
l = []
ground_pattern = re.compile(r'.*良.*')

for i in range(len(base_df_pre)):
    if ground_pattern.search(base_df_pre.loc[i, "馬場状態"]):
        l.append(np.nan)
    else:
        if base_df_pre.loc[i, "種別"] == "芝":
            l.append("芝道悪")
        elif base_df_pre.loc[i, "種別"] == "ダート":
            l.append("ダ道悪")
        else:
            l.append(np.nan)

base_df_pre["道悪判定"] = l

C:\Users\yf_te\AppData\Local\Temp\ipykernel_1492\386309958.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df_pre["道悪判定"] = l


### 血統データ

In [19]:
# カラム名を付与
peds_df_pre.columns = ["馬名", "種牡馬名"]
peds_df_pre

,馬名,種牡馬名
0,アンジュブルー,ダノンスマッシュ
1,ハングローズ,ドレフォン
2,ブーケドリス,ロードカナロア
3,セントリアン,アジアエクスプレス
4,シュガームーン,エポカドーロ
...,...,...
515,シュシュトディエス,シルバーステート
516,ホシニネガイヲ,ミスターメロディ
517,ヴァイタルピース,Pivotal
518,ナンナンプー,フィエールマン


## データの結合

In [20]:
base_df_preprocessed = base_df_pre.merge(peds_df_pre, on="馬名")

In [21]:
base_df_preprocessed = base_df_preprocessed.merge(zensou_df_pre, on="馬名")

In [22]:
base_df_preprocessed

,場所,R,クラス,種別,距離,馬場状態,馬番,馬名,年齢,騎手名,調教師名,距離区分,回り,道悪判定,種牡馬名,臨戦過程,距離変遷
0,中山,1,未勝利,ダート,1200,重(暫定),1,アンジュブルー,3歳,杉原誠人,稲垣幸雄,短距離,右回り,ダ道悪,ダノンスマッシュ,NaN,距離短縮
1,中山,1,未勝利,ダート,1200,重(暫定),2,ハングローズ,3歳,佐々木大,奥村武,短距離,右回り,ダ道悪,ドレフォン,NaN,距離短縮
2,中山,1,未勝利,ダート,1200,重(暫定),3,ブーケドリス,3歳,三浦皇成,宮田敬介,短距離,右回り,ダ道悪,ロードカナロア,休明初戦,距離短縮
3,中山,1,未勝利,ダート,1200,重(暫定),4,セントリアン,3歳,原田和真,萱野浩二,短距離,右回り,ダ道悪,アジアエクスプレス,休明初戦,距離延長
4,中山,1,未勝利,ダート,1200,重(暫定),5,シュガームーン,3歳,佐藤翔馬,的場均,短距離,右回り,ダ道悪,エポカドーロ,休明初戦,距離短縮
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,小倉,12,自己条件,芝,1200,稍(暫定),14,シュシュトディエス,5歳以上,河原田菜,石栗龍彦,短距離,右回り,芝道悪,シルバーステート,休明2走,距離延長
516,小倉,12,自己条件,芝,1200,稍(暫定),15,ホシニネガイヲ,4歳,今村聖奈,竹内正洋,短距離,右回り,芝道悪,ミスターメロディ,NaN,NaN
517,小倉,12,自己条件,芝,1200,稍(暫定),16,ヴァイタルピース,5歳以上,西塚洸二,吉田直弘,短距離,右回り,芝道悪,Pivotal,休明初戦,距離短縮
518,小倉,12,自己条件,芝,1200,稍(暫定),17,ナンナンプー,4歳,舟山瑠泉,水野貴広,短距離,右回り,芝道悪,フィエールマン,NaN,NaN


## ファイル出力

In [23]:
base_df_preprocessed.to_csv(DATA_DIR / f"preprocessed_data_{kaisai_date}.csv", index=False)